In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics
from sklearn.svm import SVC
from scipy import stats

iris = datasets.load_iris()

X_iris = iris.data

y_iris = iris.target

In [2]:
# Separar os dados para implementação do svm
y = list(map(lambda x: -1 if x == 0 else (1 if x == 1 else None) , y_iris))
y = list(filter(lambda x: x != None, y))

X = X_iris[:len(y)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [3]:
#Definição de funções necessárias

# Definir a função do kernel -> RBF
def K(x, u, gamma):
    k = np.exp(-gamma*((np.linalg.norm(x-u))**2))
    return k

#Define a função de descisão do modelo
def decision(a, y, x, u, sigma, b):
    sum = 0
    for i in range(len(a)):
        sum = sum + a[i]*y[i]*K(x[i], u, sigma)
    f = sum + b

    if f >= 0:
        return 1
    else:
        return -1

#Define a optimização para encontrar os suporte vector
def optimizacao_alfa(C):
    a = np.zeros((1, len(y_train)))
    a = a.ravel()
    firstTarget = np.random.randint(len(y_train))

    a[firstTarget] = C

    if y_train[firstTarget] == 1:
        secondTarget = np.random.randint(len(y_train))
        while y_train[secondTarget] != -1:
            secondTarget = np.random.randint(len(y_train))
        a[secondTarget] = C
    elif y_train[firstTarget] == -1:
        secondTarget = np.random.randint(len(y_train))
        while y_train[secondTarget] != 1:
            secondTarget = np.random.randint(len(y_train))
        a[secondTarget] = C
    return a


In [4]:
#Teste simples de funcionamento para um classificar com 2 classes
Y_pred = []
a = optimizacao_alfa(10)

for i in range(len(X_test)):
    y_pred = decision(a, y_train, X_train, X_test[i], 1, 1)
    Y_pred.append(y_pred)

print(f'Teste: {y_test}')
print(f'Previsão: {Y_pred}')

accuracy = metrics.accuracy_score(y_test, Y_pred)

print(f'Accuracy: {accuracy}')


Teste: [1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, -1, -1]
Previsão: [1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, -1, -1]
Accuracy: 1.0


In [5]:
#One-vs-one classifier method

X_train, X_test, Y_train, Y_test = train_test_split(X_iris, y_iris, test_size=0.3)

#Pega os valores das classes no dataset em estudo
y_set = list(set(Y_train))

#Define os duelos entre as classes sem permetir repetição
setClasses = []
for i in range(len(y_set)-1):
    for j in range(i+1,len(y_set)):
        setClasses.append([y_set[i], y_set[j]])

#Para cada duelo, iremos mapear os valores da classe como -1 e 1

res = []
for s in setClasses:
    x_train = X_train.copy().tolist()
    y_train = list(map(lambda x: -1 if x == s[0] else (1 if x == s[1] else None) , Y_train))
    for i in range(len(y_train)):
        if y_train[i] == None:
            x_train[i] = None
    y_train = list(filter(lambda x: x != None, y_train))
    x_train = list(filter(lambda i: i != None, x_train))

    #Com isso podemos rodar encontrar a classificação para cada classe:

    Y_pred = []
    a = optimizacao_alfa(10)

    for i in range(len(X_test)):
        y_pred = decision(a, y_train, x_train, X_test[i], 1, 1)
        Y_pred.append(y_pred)

    print(f'Classe = {str(s)}:')

    Y_pred = list(map(lambda x: s[0] if x == -1 else (s[1] if x == 1 else None) , Y_pred))

    print(f'\tPrevisão: {Y_pred}')

    res.append(Y_pred)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Classe = [0, 1]:
	Previsão: [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Classe = [0, 2]:
	Previsão: [0, 0, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 0, 0]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0. 

In [6]:
#Obtendo a previsão para cada duelo, temos que achar a melhor que mais se repete:

res = np.array(res)
y_pred = stats.mode(res)[0].ravel()

print(f'Previsão: {y_pred}')
print(f'Teste: {Y_test}')
y_pred = np.array(y_pred)
Y_test = np.array(Y_test)

accuracy = metrics.accuracy_score(Y_test, y_pred)

print(f'\tAccuracy: {accuracy}')

Previsão: [0 0 2 1 0 0 2 1 1 0 1 1 1 2 1 1 1 2 2 1 0 0 1 0 1 2 1 2 2 1 0 2 1 0 2 1 0
 1 1 0 1 0 2 0 0]
Teste: [0 0 1 1 0 0 2 1 1 0 1 1 1 2 1 1 1 2 2 2 0 0 1 0 1 2 2 2 1 1 0 2 2 0 2 2 0
 2 1 0 1 0 2 0 0]
	Accuracy: 0.8444444444444444


In [7]:
print(a)


[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.  0.  0.  0.  0.  0.  0.
  0. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
